In [2]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 17644



In [ ]:
sas.submitLST(f"""
/* Specify the start and end dates */
%let start_date = '202201';
%let end_date = '202212';

/* Create tables based on yyyymm keys */
%macro create_tables;
  %local yyyymm;
  
  %do yyyymm = &start_date %to &end_date;
    /* Create table for yyyymm */
    data table_&yyyymm;
        set  sashelp.class;
        /* creat a month column */
        month = &yyyymm;
    run;
  %end;
%mend;

%create_tables;

/* combines list of tables covering a range of suffixes */
%macro combine_tables;
  %local yyyymm;
  
    data combined_table;
        set 
        %do yyyymm = &start_date %to &end_date;
            table_&yyyymm
        %end;
    run;

%mend;

%combine_tables;

run;
""")

In [ ]:
sas.submitLST(f"""
/* Define the names of the tables */
%let tables = table1 table2 table3;

/* Create an empty results table */
proc sql;
    create table results (TableName char(10), SumValue num);
quit;

/* Loop over the tables */
%macro sum_tables;
    %do i = 1 %to %sysfunc(countw(&tables.));
        %let table = %scan(&tables., &i.);
        proc sql;
            /* Calculate the sum */
            create table temp as
            select sum(column_name) as SumValue
            from &table.;
            
            /* Add the result to the results table */
            insert into results
            select "&table." as TableName, SumValue
            from temp;
        quit;
        
        /* Delete the temporary table */
        proc datasets library=work nolist;
            delete temp;
        quit;
    %end;
%mend sum_tables;

%sum_tables;


run;
""")